In [ ]:
from workshop.dataset import CUBDataset
from torch.utils.data import DataLoader
from workshop.model import BirdNet

In [ ]:
dataset = CUBDataset(
        root_directory=args.datapath,
        set_="train",
        transforms=Compose([
            RandomCrop((224, 224), pad_if_needed=True),
            RandomHorizontalFlip(),
            ToTensor()
        ])
    )
data_loader = DataLoader(
    dataset,
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=args.number_workers)

model = BirdNet(num_classes=dataset.number_classes).to(args.device)


In [ ]:
print("derp")